In [21]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from lifelines import KaplanMeierFitter, CoxPHFitter
from sklearn.model_selection import train_test_split
from lifelines.statistics import logrank_test, pairwise_logrank_test
from sksurv.ensemble import RandomSurvivalForest
from sksurv.metrics import concordance_index_censored, cumulative_dynamic_auc

In [2]:
# Lendo os dados
data = pd.read_csv('../data/processed/Processed_Telecom_Data.csv')

Levando em conta os nossos dados, podemos inferir o seguinte:
- Temos dados censurados à esquerda, pois não sabemos quando o nosso 
relacionamento com os clientes começou.
- Também temos dados censorados à direita, representados pelos clientes que
ainda não cancelaram o serviço.

Como primeiro passo, vamos aplicar o teste de Kepler-Meier para estimar a
função de sobrevivência dos nossos dados. Para tal, iremos usar apenas os 
usuários que não deram churn, já que o teste fiunciona bem em dados censurados à
direita. Como esse é um teste não paramétrico, não precisamos nos preocupar com
a distribuição dos dados. Além disso, só podemos usar duas variáveis, uma para
o tempo e outra para o evento, que no nosso caso é o cancelamento do serviço.
Para tal, usaremos a coluna `tenure` e a coluna `Churn`, respectivamente.

### Teste de Kepler-Meier

In [3]:
# Inicializando o método
kmf = KaplanMeierFitter()

# Ajustando os dados
kmf.fit(data['tenure'], data['Churn'])

# Plotando o gráfico com o plotly
fig = go.Figure()
fig.add_trace(go.Scatter(x=kmf.survival_function_.index,
                         y=kmf.survival_function_['KM_estimate'],
                         mode='lines',
                         name='KM estimate'))

# Atualizando o layout da figure
fig.update_layout(
    title='<b>Probabilidade de Sobrevivência dado o tempo de relacionamento<br>com a empresa</b><br><sup><i>Análise de probabilidade dos coeficiente de Keplen-Meier</i></sup>',
    title_font=dict(size=20, family="Arial"),
    font_color='#646369',
    template='plotly_white',
    yaxis=dict(showticklabels=True, showgrid=True, title='Probabilidade de Sobrevivência'),
    xaxis=dict(showgrid=False, title='Tempo em meses'),
    title_x=0.1,
    width=910,
    height=500,
    showlegend=False
)

A probabilidade geral de sobrevivência é bem alta, se mantendo acima de 50% 
durante todo o período. Nos primeiros 4 meses, a probabilidade do cliente não 
vivenciar o evento de interesse (Churn) é acima de 90%.

Agora, vamos ver se há diferença entre os grupos de gênero e senioridade.

In [4]:
# Inicializando o método
kmf = KaplanMeierFitter()

# Initializando a figure
fig = go.Figure()

# Ajustando os dados
for genero in data['gender'].unique():
    data_gender = data.query(f"gender == '{genero}'")
    kmf.fit(data_gender['tenure'], data_gender['Churn'], label=genero)
    fig.add_trace(go.Scatter(x=kmf.survival_function_.index,
                                y=kmf.survival_function_[f'{genero}'],
                                mode='lines',
                                name=genero))
    
# Atualizando o layout da figure
fig.update_layout(
    title='<b>Probabilidade de Sobrevivência dado o tempo de relacionamento<br>com a empresa</b><br><sup><i>Análise de probabilidade dos coeficiente de Keplen-Meier para os diferentes gêneros</i></sup>',
    title_font=dict(size=20, family="Arial"),
    font_color='#646369',
    template='plotly_white',
    yaxis=dict(showticklabels=True, showgrid=True, title='Probabilidade de Sobrevivência'),
    xaxis=dict(showgrid=False, title='Tempo em meses'),
    title_x=0.1,
    width=910,
    height=500,
    showlegend=True
)

In [5]:
# Dividindo os dados por gênero 
data_masculino = data.query("gender == 'Male'")
data_feminino = data.query("gender == 'Female'")

# Aplicando o teste
resultados = logrank_test(data_masculino['tenure'], data_feminino['tenure'], data_masculino['Churn'], data_feminino['Churn'])

if resultados.p_value < 0.05:
    print(f"Com um p valor de {resultados.p_value}, podemos rejeitar a hipótese nula de que os grupos são significativamente iguais")
    
else:
    print(f"Com um p valor de {resultados.p_value}, não podemos rejeitar a hipótese nula de que os grupos são significativamente iguais")

Com um p valor de 0.46841735075696966, não podemos rejeitar a hipótese nula de que os grupos são significativamente iguais


No geral, pessoas do gênero masculino têm uma probabilidade de churn maior do
que pessoas do gênero feminino. No entanto, a diferença não é tão grande. Também 
é possível notar que o cenário se inverte com clientes com um tempo de 
relacionamento superior a 67 meses, com as pessoas do gênero feminino tendo uma
probabilidade de churn maior do que as pessoas do gênero masculino.

Com 95% de confiança, podemos dizer que as curvas de churn para os grupos de
gênero são iguais.

In [6]:
# Inicializando o método
kmf = KaplanMeierFitter()

# Initializando a figure
fig = go.Figure()

# Ajustando os dados
for senioridade in data['SeniorCitizen'].unique():
    data_senior = data.query(f"SeniorCitizen == {senioridade}")
    if senioridade == 1:
        label = 'Senior'
        kmf.fit(data_senior['tenure'], data_senior['Churn'], label=label)
        fig.add_trace(go.Scatter(x=kmf.survival_function_.index,
                                y=kmf.survival_function_[f'{label}'],
                                mode='lines',
                                name=label))
    else:
        label = 'Not senior'
        kmf.fit(data_senior['tenure'], data_senior['Churn'], label=label)
        fig.add_trace(go.Scatter(x=kmf.survival_function_.index,
                                y=kmf.survival_function_[f'{label}'],
                                mode='lines',
                                name=label))
    
# Atualizando o layout da figure
fig.update_layout(
    title='<b>Probabilidade de Sobrevivência dado o tempo de relacionamento<br>com a empresa</b><br><sup><i>Análise de probabilidade dos coeficiente de Keplen-Meier para os diferentes grupos de idade</i></sup>',
    title_font=dict(size=20, family="Arial"),
    font_color='#646369',
    template='plotly_white',
    yaxis=dict(showticklabels=True, showgrid=True, title='Probabilidade de Sobrevivência'),
    xaxis=dict(showgrid=False, title='Tempo em meses'),
    title_x=0.1,
    width=910,
    height=500,
    showlegend=True
)

In [7]:
# Obtendo os dados de não senior
data_not_senior = data.query("SeniorCitizen == 0")
data_senior = data.query("SeniorCitizen == 1")

# Aplicando o teste
resultados = logrank_test(data_senior['tenure'], data_not_senior['tenure'], data_senior['Churn'], data_not_senior['Churn'])

if resultados.p_value < 0.05:
    print(f"Com um p valor de {resultados.p_value}, podemos rejeitar a hipótese nula de que os grupos são significativamente iguais")
    
else:
    print(f"Com um p valor de {resultados.p_value}, não podemos rejeitar a hipótese nula de que os grupos são significativamente iguais")

Com um p valor de 1.2676192066669992e-25, podemos rejeitar a hipótese nula de que os grupos são significativamente iguais


Pessoas mais velhas são mais propensas a dar churn do que pessoas mais novas.
A diferença durante o último mês do período chegou a ser de 20%. 

Com 95% de confiança, podemos dizer que as curvas de churn para os grupos de 
senioridade são significativamente diferentes.

Agora, vamos considerar as variáveis 'Partner' e 'Dependents', para averiguar se
há diferença entre pessoas que moram sozinhas e pessoas que moram com alguém.

In [8]:
# Inicializando o método
kmf = KaplanMeierFitter()

# Initializando a figure
fig = go.Figure()

# Ajustando os dados
for resposta in data['Partner'].unique():
    data_partner = data.query(f"Partner == '{resposta}'")
    kmf.fit(data_partner['tenure'], data_partner['Churn'], label=resposta)
    fig.add_trace(go.Scatter(x=kmf.survival_function_.index,
                                y=kmf.survival_function_[f'{resposta}'],
                                mode='lines',
                                name=resposta))
    
# Atualizando o layout da figure
fig.update_layout(
    title='<b>Probabilidade de Sobrevivência dado o tempo de relacionamento<br>com a empresa</b><br><sup><i>Análise de probabilidade dos coeficiente de Keplen-Meier para pessoas com parceiros</i></sup>',
    title_font=dict(size=20, family="Arial"),
    font_color='#646369',
    template='plotly_white',
    yaxis=dict(showticklabels=True, showgrid=True, title='Probabilidade de Sobrevivência'),
    xaxis=dict(showgrid=False, title='Tempo em meses'),
    title_x=0.1,
    width=910,
    height=500,
    showlegend=True
)

In [9]:
# Dividindo os dados por gênero 
data_partner_yes = data.query("Partner == 'Yes'")
data_partner_no = data.query("Partner == 'No'")

# Aplicando o teste
resultados = logrank_test(data_partner_yes['tenure'], data_partner_no['tenure'], data_partner_yes['Churn'], data_partner_no['Churn'])

if resultados.p_value < 0.05:
    print(f"Com um p valor de {resultados.p_value}, podemos rejeitar a hipótese nula de que os grupos são significativamente iguais")
    
else:
    print(f"Com um p valor de {resultados.p_value}, não podemos rejeitar a hipótese nula de que os grupos são significativamente iguais")

Com um p valor de 4.132951134427116e-94, podemos rejeitar a hipótese nula de que os grupos são significativamente iguais


A probabilidade de sobrevivência para pessoas que possuem parceiros é maior do
que para pessoas que não possuem parceiros. A diferença é de aproximadamente
20% durante boa parte do período.

Com 95% de confiança, podemos dizer que as curvas de churn para os grupos de
parceiros são significativamente diferentes.

In [10]:
# Inicializando o método
kmf = KaplanMeierFitter()

# Initializando a figure
fig = go.Figure()

# Ajustando os dados
for resposta in data['Dependents'].unique():
    data_dependents = data.query(f"Dependents == '{resposta}'")
    kmf.fit(data_dependents['tenure'], data_dependents['Churn'], label=resposta)
    fig.add_trace(go.Scatter(x=kmf.survival_function_.index,
                                y=kmf.survival_function_[f'{resposta}'],
                                mode='lines',
                                name=resposta))
    
# Atualizando o layout da figure
fig.update_layout(
    title='<b>Probabilidade de Sobrevivência dado o tempo de relacionamento<br>com a empresa</b><br><sup><i>Análise de probabilidade dos coeficiente de Keplen-Meier para pessoas com dependentes</i></sup>',
    title_font=dict(size=20, family="Arial"),
    font_color='#646369',
    template='plotly_white',
    yaxis=dict(showticklabels=True, showgrid=True, title='Probabilidade de Sobrevivência'),
    xaxis=dict(showgrid=False, title='Tempo em meses'),
    title_x=0.1,
    width=910,
    height=500,
    showlegend=True
)

In [11]:
# Dividindo os dados por gênero 
data_dependents_yes = data.query("Dependents == 'Yes'")
data_dependents_no = data.query("Dependents == 'No'")

# Aplicando o teste
resultados = logrank_test(data_dependents_yes['tenure'], data_dependents_no['tenure'], data_dependents_yes['Churn'], data_dependents_no['Churn'])

if resultados.p_value < 0.05:
    print(f"Com um p valor de {resultados.p_value}, podemos rejeitar a hipótese nula de que os grupos são significativamente iguais")
    
else:
    print(f"Com um p valor de {resultados.p_value}, não podemos rejeitar a hipótese nula de que os grupos são significativamente iguais")

Com um p valor de 1.5372382701499682e-52, podemos rejeitar a hipótese nula de que os grupos são significativamente iguais


A probabilidade de sobvrevivência de pessoas com dependentes é maior
do que para pessoas sem dependentes. A diferença durante o último mês do período
chegou a pouco menos de 25%.

Com 95% de confiança, podemos dizer que as curvas de churn para os grupos de
dependentes são significativamente diferentes.

Por fim, vamos analisar como a probabilidade de sobrevivência varia de acordo
com o tipo de contrato.

In [12]:
# Inicializando o método
kmf = KaplanMeierFitter()

# Initializando a figure
fig = go.Figure()

# Ajustando os dados
for tipo_contrato in data['Contract'].unique():
    data_contract = data.query(f"Contract == '{tipo_contrato}'")
    kmf.fit(data_contract['tenure'], data_contract['Churn'], label=tipo_contrato)
    fig.add_trace(go.Scatter(x=kmf.survival_function_.index,
                                y=kmf.survival_function_[f'{tipo_contrato}'],
                                mode='lines',
                                name=tipo_contrato))
    
# Atualizando o layout da figure
fig.update_layout(
    title='<b>Probabilidade de Sobrevivência dado o tempo de relacionamento<br>com a empresa</b><br><sup><i>Análise de probabilidade dos coeficiente de Keplen-Meier para pessoas com dependentes</i></sup>',
    title_font=dict(size=20, family="Arial"),
    font_color='#646369',
    template='plotly_white',
    yaxis=dict(showticklabels=True, showgrid=True, title='Probabilidade de Sobrevivência'),
    xaxis=dict(showgrid=False, title='Tempo em meses'),
    title_x=0.1,
    width=910,
    height=500,
    showlegend=True
)

In [13]:
# Aplicando o teste
resultados = pairwise_logrank_test(data['tenure'], data['Contract'], data['Churn'])

# Buscando o menor p valor
p_valor = resultados.p_value.min()

if p_valor < 0.05:
    print(f"Com um p valor de {p_valor}, podemos rejeitar a hipótese nula de que os grupos são significativamente iguais")

else:
    print(f"Com um p valor de {p_valor}, não podemos rejeitar a hipótese nula de que os grupos são significativamente iguais")

Com um p valor de 0.0, podemos rejeitar a hipótese nula de que os grupos são significativamente iguais


Contratos mensais têm uma probabilidade de sobrevivência menor do que contratos
anuais. A probabilidade de sobrevivência ficou abaixo de 50% no trigésimo quinto
mês para contratos mensais.

Com 95% de confiança, podemos dizer que pelo menos uma das curvas de churn para 
os grupos de contrato é significativamente diferente.

### Treinando um modelo multivariado
Até aqui, usamos apenas uma variável para estimar a probabilidade de 
sobrevivência. Agora, vamos usar uma abordagem multivariada, usando todas as
variáveis disponíveis.

In [14]:
# Criando 2 grupos estratificados para treino e teste
treino, teste = train_test_split(data, 
                               test_size=0.2, 
                               random_state=200, 
                               stratify=data['Churn'])

# Selecionando as variáveis categóricas
cat_cols = treino.select_dtypes(include='object').columns

# Criando um pipeline para as variáveis categóricas
cat_pipe = Pipeline([('onehot', OneHotEncoder(drop='first'))])

# Criando um transformer para as variáveis categóricas
transformer = ColumnTransformer([('cat', cat_pipe, cat_cols)], remainder='passthrough')

In [15]:
# Transformando os dados de treino
treino_transformado = transformer.fit_transform(treino)

# Buscando os nomes das colunas
colunas = transformer.get_feature_names_out()

# Transformando os dados de teste
teste_transformado = transformer.transform(teste)

# Criando um dataframe com os dados transformados
treino_transformado = pd.DataFrame(treino_transformado, columns=colunas)
teste_transformado = pd.DataFrame(teste_transformado, columns=colunas)

In [16]:
# Instanciando o modelo
modelo = CoxPHFitter(penalizer=0.01)

# Ajustando o modelo
modelo.fit(treino_transformado, 
           duration_col='remainder__tenure', 
           event_col='remainder__Churn')

/home/daniel/Documents/survival_analysis/.venv/lib/python3.10/site-packages/lifelines/utils/__init__.py:1185: UserWarning:

Attempting to convert an unexpected datatype 'object' to float. Suggestion: 1) use `lifelines.utils.datetimes_to_durations` to do conversions or 2) manually convert to floats/booleans.



<lifelines.CoxPHFitter: fitted with 5634 total observations, 4139 right-censored observations>

In [17]:
# Checando o c-index do treino
modelo.score(treino_transformado, 
             scoring_method='concordance_index')


0.9234288939914063

In [18]:
# Checando o c-index do teste
modelo.score(teste_transformado, 
             scoring_method='concordance_index')

0.9286198322034427

In [20]:
# Dividindo os dados de treino
x_treino = treino_transformado.drop(['remainder__tenure', 'remainder__Churn'], 
                                    axis=1)
y_treino = treino_transformado[['remainder__Churn', 
                                'remainder__tenure']].to_records(index=False, 
                                                                column_dtypes={'remainder__Churn': 'bool',
                                                                               'remainder__tenure': 'int64'})

# Inicializando o modelo
modelo_forest = RandomSurvivalForest(random_state=200)

# Treinando o modelo
modelo_forest.fit(x_treino, y_treino)

# Calculando o c-index
concordance_index_censored(y_treino['remainder__Churn'], y_treino['remainder__tenure'], modelo_forest.predict(x_treino))


(0.935139504171267, 5337633, 370204, 21, 129030)

In [40]:
# Calculando o c-index do teste
x_teste = teste_transformado.drop(['remainder__tenure', 'remainder__Churn'], 
                                    axis=1)
y_teste = teste_transformado[['remainder__Churn',
                                'remainder__tenure']].to_records(index=False, 
                                                                    column_dtypes={'remainder__Churn': 'bool',
                                                                                 'remainder__tenure': 'int64'})
                                
concordance_index_censored(y_teste['remainder__Churn'], y_teste['remainder__tenure'], modelo_forest.predict(x_teste))

(0.9046468701887499, 319059, 33630, 0, 7503)